In [1]:
import os
import pickle
import numpy as np
import tensorflow as tf

# Import project modules
from conllu_reader import ConlluReader
from algorithm import ArcEager, Transition, Sample
from conllu_token import Token
from model import ParserMLP

# Check for GPU availability (Optional)
print("TensorFlow Version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2025-12-03 17:55:45.948812: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-03 17:55:45.963590: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8473] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-03 17:55:45.968416: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1471] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-03 17:55:45.980047: I tensorflow/core/platform/cpu_feature_guard.cc:211] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow Version: 2.17.0
Num GPUs Available:  1


I0000 00:00:1764784547.583690     190 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764784547.644219     190 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764784547.644253     190 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
dataset_path = "dataset.pkl"

if not os.path.exists(dataset_path):
    print(f"Error: '{dataset_path}' not found. Please run main.py first to generate it.")
else:
    print(f"Loading dataset from {dataset_path}...")
    with open(dataset_path, "rb") as f:
        data = pickle.load(f)
        
    training_samples = data["training_samples"]
    dev_samples = data["dev_samples"]
    deprels = data["deprels"]
    actions = data["actions"]
    
    print("Dataset loaded successfully!")
    print(f"Training Samples: {len(training_samples)}")
    print(f"Development Samples: {len(dev_samples)}")
    print(f"Unique Dependency Labels: {len(deprels)}")

Loading dataset from dataset.pkl...
Dataset loaded successfully!
Training Samples: 81182
Development Samples: 4978
Unique Dependency Labels: 43


In [3]:
# Inspect the first training sample
sample = training_samples[80000]

print(f"--- Sample 1 ---")
# Note: Accessing internal state for visualization
print(f"State Stack: {[t.form for t in sample.state.S]}")
print(f"State Buffer (first 3): {[t.form for t in sample.state.B[:3]]}...")
print(f"Gold Transition: {sample.transition}")

--- Sample 1 ---
State Stack: ['ROOT', 'The']
State Buffer (first 3): ['revolution', 'in', 'the']...
Gold Transition: LEFT-ARC-det


In [4]:
# Extract features from the loaded sample
feats = sample.state_to_feats(nstack_feats=1, nbuffer_feats=1)

print("Feature List:", feats)

# Expected output format: 
# [Stack_Word_1, Stack_Word_0, Buffer_Word_0, Buffer_Word_1, 
#  Stack_UPOS_1, Stack_UPOS_0, Buffer_UPOS_0, Buffer_UPOS_1]

Feature List: ['The', 'revolution', 'DET', 'NOUN']


In [5]:
# Initialize Model
# You can adjust dimensions and epochs as needed
model = ParserMLP(word_emb_dim=100, hidden_dim=64, epochs=3, batch_size=32)

print("Starting training...")
# The train function handles vocabulary building and vectorization internally
model.train(training_samples, dev_samples)

Starting training...
Building vocabulary...
Vocab built: 6872 words, 20 UPOS tags.
Detected Feature Shape: 4 words, 4 tags.
Vectorizing data...
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 words_input (InputLayer)    [(None, 4)]                  0         []                            
                                                                                                  
 upos_input (InputLayer)     [(None, 4)]                  0         []                            
                                                                                                  
 embedding (Embedding)       (None, 4, 100)               687200    ['words_input[0][0]']         
                                                                                                  
 embedding_1 (Embedding)     (None, 4, 50)       

I0000 00:00:1764784549.076250     190 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764784549.076302     190 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764784549.076312     190 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764784549.497826     190 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764784549.497879     190 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-12-03

Epoch 1/3


'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring f

   1/2537 [..............................] - ETA: 27:10:31 - loss: 4.1982 - action_output_loss: 1.3797 - deprel_output_loss: 3.7580 - action_output_accuracy: 0.2812 - deprel_output_accuracy: 0.0000e+00

...

2537/2537 [==============================] - 32s 13ms/step - loss: 0.7242 - action_output_loss: 0.3145 - deprel_output_loss: 0.5462 - action_output_accuracy: 0.8861 - deprel_output_accuracy: 0.8281 - val_loss: 0.8267 - val_action_output_loss: 0.3951 - val_deprel_output_loss: 0.5755 - val_action_output_accuracy: 0.8620 - val_deprel_output_accuracy: 0.8268
Epoch 3/3
 445/2537 [====>.........................] - ETA: 3:25 - loss: 0.5472 - action_output_loss: 0.2259 - deprel_output_loss: 0.4284 - action_output_accuracy: 0.9216 - deprel_output_accuracy: 0.8617

'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)


1423/2537 [===============>..............] - ETA: 49s - loss: 0.5652 - action_output_loss: 0.2374 - deprel_output_loss: 0.4371 - action_output_accuracy: 0.9165 - deprel_output_accuracy: 0.8578

'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)


2537/2537 [==============================] - 52s 21ms/step - loss: 0.5712 - action_output_loss: 0.2432 - deprel_output_loss: 0.4374 - action_output_accuracy: 0.9128 - deprel_output_accuracy: 0.8581 - val_loss: 0.8537 - val_action_output_loss: 0.4206 - val_deprel_output_loss: 0.5775 - val_action_output_accuracy: 0.8640 - val_deprel_output_accuracy: 0.8282


In [6]:
from conllu_reader import ConlluReader

# Initialize reader if not already done
reader = ConlluReader()

# Load the test set
# We use inference=True to ensure we treat this as unparsed data
print("Loading test set...")
test_trees = reader.read_conllu_file("en_partut-ud-test_clean.conllu", inference=True)
print(f"Loaded {len(test_trees)} sentences for testing.")

Loading test set...
Loaded 153 sentences for testing.


In [7]:
test_trees[1]

[0	ROOT	ROOT	ROOT_UPOS	ROOT_CPOS	ROOT_FEATS	_	_	_	_,
 1	Any	any	DET	DI	PronType=Ind	_	_	_	_,
 2	use	use	NOUN	S	Number=Sing	_	_	_	_,
 3	of	of	ADP	E	_	_	_	_	_,
 4	the	the	DET	RD	Definite=Def|PronType=Art	_	_	_	_,
 5	work	work	NOUN	S	Number=Sing	_	_	_	_,
 6	other	other	ADJ	A	Degree=Pos	_	_	_	_,
 7	than	than	SCONJ	CS	_	_	_	_	_,
 8	as	as	ADP	E	_	_	_	_	_,
 9	authorized	authorize	VERB	V	Tense=Past|VerbForm=Part	_	_	_	_,
 10	under	under	ADP	E	_	_	_	_	_,
 11	this	this	DET	DD	Number=Sing|PronType=Dem	_	_	_	_,
 12	license	license	NOUN	S	Number=Sing	_	_	_	_,
 13	or	or	CCONJ	CC	_	_	_	_	_,
 14	copyright	copyright	NOUN	S	Number=Sing	_	_	_	_,
 15	law	law	NOUN	S	Number=Sing	_	_	_	_,
 16	is	be	AUX	VA	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	_	_	_	_,
 17	prohibited	prohibit	VERB	V	Tense=Past|VerbForm=Part	_	_	_	_,
 18	.	.	PUNCT	FS	_	_	_	_	_]

In [8]:
# Conduct inference on the test set
print("Running inference on the test set...")
# The model.run method modifies the trees in-place or returns them
# It predicts the HEAD and DEPREL for each token
predicted_test_trees = model.run(test_trees)

print("Inference complete.")

Running inference on the test set...
Inference complete.


In [9]:
output_path = "output_one_feature_raw.conllu"

print(f"Saving raw predictions to {output_path}...")
reader.write_conllu_file(output_path, predicted_test_trees)
print("File saved.")

Saving raw predictions to output_one_feature_raw.conllu...
File saved.


In [10]:
from postprocessor import PostProcessor

post = PostProcessor()

print(f"Post-processing predictions in {output_path}...")

fixed_trees = post.postprocess(output_path)

final_output_path = "output_one_feature_fixed.conllu"
reader.write_conllu_file(final_output_path, fixed_trees)

print(f"Post-processing complete. Final predictions saved to {final_output_path}")

Post-processing predictions in output_one_feature_raw.conllu...
Post-processing complete. Final predictions saved to output_one_feature_fixed.conllu


In [12]:
# Run the evaluation script comparing the Gold Standard (test_clean) against your Fixed Output
# -v provides verbose output
!python conll18_ud_eval.py en_partut-ud-test_clean.conllu output_one_feature_fixed.conllu -v

Metric     | Precision |    Recall |  F1 Score | AligndAcc
-----------+-----------+-----------+-----------+-----------
Tokens     |    100.00 |    100.00 |    100.00 |
Sentences  |    100.00 |    100.00 |    100.00 |
Words      |    100.00 |    100.00 |    100.00 |
UPOS       |    100.00 |    100.00 |    100.00 |    100.00
XPOS       |    100.00 |    100.00 |    100.00 |    100.00
UFeats     |    100.00 |    100.00 |    100.00 |    100.00
AllTags    |    100.00 |    100.00 |    100.00 |    100.00
Lemmas     |    100.00 |    100.00 |    100.00 |    100.00
UAS        |     76.50 |     76.50 |     76.50 |     76.50
LAS        |     66.02 |     66.02 |     66.02 |     66.02
CLAS       |     51.08 |     50.36 |     50.72 |     50.36
MLAS       |     49.72 |     49.02 |     49.36 |     49.02
BLEX       |     51.08 |     50.36 |     50.72 |     50.36
